In [3]:
%load_ext autoreload


In [4]:
%autoreload 1


In [5]:
from transformers import AutoTokenizer, LlamaConfig, LlamaTokenizer,LlamaForCausalLM, LlamaTokenizer,TextDataset,DataCollatorForLanguageModeling
from peft import get_peft_config, get_peft_model, PeftModel, PeftConfig ,LoraConfig,TaskType
from transformers import TrainingArguments, Trainer,GenerationConfig,LineByLineTextDataset
from datasets import Dataset , load_dataset
import torch
import textwrap
from torch.nn import CrossEntropyLoss
import numpy as np
import json
import pickle
from transformers import StoppingCriteria, StoppingCriteriaList
import re
%aimport vicuna_setup
from vicuna_pubmedqa_eval import model_pubmed_evaluator
import pandas as pd
import csv
from collections import Counter
from evaluation import pub_scores


/home/minigpt_env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /home/minigpt_env/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cuda113.so
CUDA SETUP: CUDA runtime path found: /home/minigpt_env/lib/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 8.0
CUDA SETUP: Detected CUDA version 113
CUDA SETUP: Loading binary /home/minigpt_env/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cuda113.so...


/home/minigpt_env/lib/python3.9/site-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: Found duplicate ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] files: {PosixPath('/home/minigpt_env/lib/libcudart.so'), PosixPath('/home/minigpt_env/lib/libcudart.so.11.0')}.. We'll flip a coin and try one of these, in order to fail forward.
Either way, this might cause trouble in the future:
If you get `CUDA error: invalid device function` errors, the above might be the cause and the solution is to make sure only one ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] in the paths that we search based on your env.
  warn(msg)


In [6]:
#torch.cuda.empty_cache()

In [7]:
model_path="/home/vicuna-weights-7B"
peft_path="/home/medical-llama/vicuna-lora-7B"
tokenizer = LlamaTokenizer.from_pretrained(model_path,clean_up_tokenization_spaces=True,padding=False)

In [8]:
#tokenizer settings

In [9]:
#with open("/home/medical-llama/vicuna-lora-7B/adapter_config.json", 'r') as json_file:
#    lora_config = json.load(json_file)

In [10]:
torch.set_default_tensor_type(torch.cuda.HalfTensor)
model = LlamaForCausalLM.from_pretrained(model_path)
torch.set_default_tensor_type(torch.cuda.FloatTensor)

Loading checkpoint shards: 100%|██████████| 2/2 [00:09<00:00,  4.53s/it]


In [11]:
#peft_config = PeftConfig.from_pretrained(peft_path)

In [12]:
model = PeftModel.from_pretrained(
        model,
        peft_path,
        torch_dtype=torch.float16,
)

In [79]:
model.config.pad_token_id = tokenizer.pad_token_id = 0  # unk
model.config.bos_token_id = 1
model.config.eos_token_id = 2


In [13]:
model = model.eval()
model = torch.compile(model)

In [14]:
pubmed_config=GenerationConfig(
        temperature=0.4,
        top_p=0.75,
        repetition_penalty=1.1,
        beam_size=1,
        max_length=5,
        min_length=1,
        max_time=3,
        max_new_tokens=10, # 3
        #forced_bos_token_id=[tokenizer.encode("yes")[1], tokenizer.encode("no")[1], tokenizer.encode("maybe")[1]],
        
    )

In [15]:
df = pd.read_csv("/home/medical-llama/Pubmedqa/pubmedqa/data/pubmedqa_test.csv")

In [16]:
def pubmed_template(Question, Context):
        
        prompt_template=f"""
    Respond with "yes","no","maybe" using the context & the question provided.    
     
    Context: Research involving dogs has been instrumental in advancing our understanding of various scientific fields. Dogs are highly trainable and sociable animals.

    Question: Are dogs trainable?

    Answer: yes

    Context: Research involving dogs has been instrumental in advancing our understanding of various scientific fields. Dogs are highly trainable and sociable animals.

    Question: Can dogs be trained to detect and alert individuals to the presence of specific allergens?

    Answer: maybe

    Context: Research involving dogs has been instrumental in advancing our understanding of various scientific fields. Dogs are highly trainable and sociable animals.

    Question: Are Dogs highly introverted & reserved ?

    Answer: no 


    Context:
    {Context}

    Question:
    {Question}

    Answer: 
    
    """
        return prompt_template

In [84]:
def pubmed_template(Question, Context):
        
        prompt_template=f"""

    your task is to answering research questions relating to medical domain using yes/no/maybe responses. Answer as yes if the context supports the question , answer as no if the context does not support the question , answer as maybe if the context is not clear enough to answer the question.      

    ###
            
    Context: Research involving dogs has been instrumental in advancing our understanding of various scientific fields. Dogs are highly trainable and sociable animals.

    Question: Are dogs trainable?

    Answer: yes

    ###

    Context: Research involving dogs has been instrumental in advancing our understanding of various scientific fields. Dogs are highly trainable and sociable animals.

    Question: Can dogs be trained to detect and alert individuals to the presence of specific allergens?

    Answer: maybe

    ###

    Context: Research involving dogs has been instrumental in advancing our understanding of various scientific fields. Dogs are highly trainable and sociable animals.

    Question: Are Dogs highly introverted & reserved ?

    Answer: no 

    ###

    Context:
    {Context}

    Question:
    {Question}

    Answer: 
    
    """
        return prompt_template

In [17]:
vicuna_setup.vicuna_inference(pubmed_template(Question=str(df['Question'][5]),Context=str(df['Context'][5])),model=model,tokenizer=tokenizer,config=pubmed_config)


''

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/home/minigpt_env/lib/python3.9/site-packages/IPython/core/interactiveshell.py", line 3508, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_30693/3327623571.py", line 1, in <module>
    tokenizer.decode([-100])
  File "/home/minigpt_env/lib/python3.9/site-packages/transformers/tokenization_utils_base.py", line 3496, in decode
    return self._decode(
  File "/home/minigpt_env/lib/python3.9/site-packages/transformers/tokenization_utils.py", line 931, in _decode
    filtered_tokens = self.convert_ids_to_tokens(token_ids, skip_special_tokens=skip_special_tokens)
  File "/home/minigpt_env/lib/python3.9/site-packages/transformers/tokenization_utils.py", line 912, in convert_ids_to_tokens
    tokens.append(self._convert_id_to_token(index))
  File "/home/minigpt_env/lib/python3.9/site-packages/transformers/models/llama/tokenization_llama.py", line 129, in _convert_id_to_token
    token = self.sp_model.IdToPiec

In [ ]:
#print(pubmed_template(Question=df['Question'][0],Context=df['Context'][0]))

In [ ]:
config=GenerationConfig(
        temperature=0.4,
        top_p=0.75,
        repetition_penalty=1.1,
        beam_size=1,
        max_length=128,
        min_length=1,
        max_time=10,
        max_new_tokens=128,)


In [ ]:
print(vicuna_inference("can i take paracetamol & drink alcohol ?",model=model,tokenizer=tokenizer,config=config))


In [ ]:
ans_dict={}
for i in range(int(df.shape[0]/10)):
    pmid=str(df['ID'][i])
    ans=vicuna_inference(pubmed_template(Question=df['Question'][i],Context=df['Context'][i]),model=model,tokenizer=tokenizer,config=pubmed_config)
    ans_dict[pmid]=ans

In [ ]:
ans_dict=model_pubmed_evaluator(model=model,tokenizer=tokenizer,config=pubmed_config,id_=0)


In [17]:
# outputs of model 
tokenizer.decode([ 1, 29871,    13, 21140,   340,   338,   385, 15278,   393, 16612,
           263,  3414, 29889, 14350,   263,  2933,   393,  7128,  2486,  1614,
          2167,   278,  2009, 29889,    13, 29871,    13,  2277, 29937,  2799,
          4080, 29901,    13,    13,  1678,  2538,  2818,   411,   376,  3582,
          3284,  1217,  3284, 26026, 29908,   773,   278,  3030,   669,   278,
          1139,  4944, 29889,   268,    13,   418,    13,  1678, 15228, 29901,
         10550, 21677, 26361,   756,  1063, 11395,   284,   297,  3061, 19985,
          1749,  8004,   310,  5164, 16021,  4235, 29889,   360, 12099,   526,
         10712,  7945,   519,   322,  5374,   519, 15006, 29889,    13,    13,
          1678,   894, 29901,  4683, 26361,  7945,   519, 29973,    13,    13,
          1678,   673, 29901,  4874,    13,    13,  1678, 15228, 29901, 10550,
         21677, 26361,   756,  1063, 11395,   284,   297,  3061, 19985,  1749,
          8004,   310,  5164, 16021,  4235, 29889,   360, 12099,   526, 10712,
          7945,   519,   322,  5374,   519, 15006, 29889,    13,    13,  1678,
           894, 29901,  1815, 26361,   367, 16370,   304,  6459,   322,  6655,
         15724,   304,   278, 10122,   310,  2702,   599, 15064,   575, 29973,
            13,    13,  1678,   673, 29901,  5505,    13,    13,  1678, 15228,
         29901, 10550, 21677, 26361,   756,  1063, 11395,   284,   297,  3061,
         19985,  1749,  8004,   310,  5164, 16021,  4235, 29889,   360, 12099,
           526, 10712,  7945,   519,   322,  5374,   519, 15006, 29889,    13,
            13,  1678,   894, 29901,  4683,   360, 12099, 10712, 22909,  1765,
           287,   669, 21676,  1577,    13,    13,  1678,   673, 29901,   694,
         29871,    13,    13,    13,  1678, 15228, 29901,    13,  1678,   319,
          1206,   310,   805,   979,  1014, 29881,  3631,   298,  4579,  4125,
           313,  1799, 29928, 29950, 29897,  1494,  1014,   279,   496,  1217,
           333,  9736,   272, 19046,   482,   313,  8132, 29950, 29897,  1363,
           310,   263,  5796,   415,  2955,  7463,  1559,   327,   333,   385,
          5411,   952, 29885,   338,  5439, 29889, 10506,   263,  1206,   756,
          2360,  1063,  8967, 29889,   319, 29871, 29945, 29906, 29899,  6360,
         29899,  1025,  6114,  1090, 29893,   296,   263,   274,   661, 24414,
         16103,   363,   263,  5796,   415,  2955,  7463,  1559,   327,   333,
           385,  5411,   952, 29885, 29889,   319, 15712,  6454,  5275, 12812,
         10018,   393, 16698, 29950, 22856,   758, 24130, 10835,   297,   278,
         13446,   285, 20634,   322,  1014, 29881,  3631,   298,  4579,  4125,
         19540,   278,   274,   406, 12562,   279, 12033,   272,  1974, 29889,
           512, 26614,   354,  1052, 17517,   310,   318,   307,  9089,   559,
         29892,  6599, 17517,   310, 25995,   566,   309, 17546,   305,  5095,
           680, 29892,   322,  9126,   274,   406,  6729,  1028,   979, 22576,
           313,  9295, 29943, 29897,  3415, 22061,   362,  3025,   274,   275,
          1890,   270,  6038,   482,   892,  8560,   408,   410, 11461,   433,
         20009,  7539,  1860,   363, 19723,  4705, 11625, 29889,  1551,   278,
         25963,  1400,  3372,  1230,  2462, 29892,   278, 16500, 15313,  1312,
           310, 22261,  5224,  1250,   322,   541,   517,   384,  6788, 29889,
          3561,  1212,   293, 27396,   749,  6382,   292, 10018,   263,  1014,
         29881,  3631,   298,  4579,  4125,   297,   278,   301,  3774,   359,
           562,  1705,  5120, 29889,  8512,   278,  4158,  2779,   471, 20607,
         29892,   278, 16500, 10018,   694,   452,  2192, 19227, 25828,  4835,
           916,  1135,   278,  4560, 29288, 29889,  2296,   471, 14914,  8976,
          6703, 29889,   450,   298,  4579,  4125, 23556,  1490, 22020,   322,
           750, 22964,  3276,  6446, 29871, 29896, 29945, 11405,  2678, 29889,
          2439,  6788, 22020, 11684,  2618, 29892,   322,  1183,   471,   766,
         25389,   287, 29871, 29955, 11405,  2678,  1728,   738,   452,  2192,
         19227,   822,   293,   277, 29889,    13,    13,  1678,   894, 29901,
            13,  1678,  1706,   979,  1014, 29881,  3631,   298,  4579,  4125,
         29901,   263,  8617,  3100,   310,   263,  5796,   415,  2955,   938,
           945,   661,   616,   385,  5411,   952, 29885, 29973,    13,    13,
          1678,   673, 29901, 29871,    13,   268,    13,   268,    13, 29871,
            13,  2277, 29937, 13291, 29901,    13,    13,   259,    13])

'<s>\nBelow is an instruction that describes a task. Write a response that appropriately completes the request.\n \n### Instruction:\n\n    Respond with "yes","no","maybe" using the context & the question provided.    \n     \n    Context: Research involving dogs has been instrumental in advancing our understanding of various scientific fields. Dogs are highly trainable and sociable animals.\n\n    Question: Are dogs trainable?\n\n    Answer: yes\n\n    Context: Research involving dogs has been instrumental in advancing our understanding of various scientific fields. Dogs are highly trainable and sociable animals.\n\n    Question: Can dogs be trained to detect and alert individuals to the presence of specific allergens?\n\n    Answer: maybe\n\n    Context: Research involving dogs has been instrumental in advancing our understanding of various scientific fields. Dogs are highly trainable and sociable animals.\n\n    Question: Are Dogs highly introverted & reserved ?\n\n    Answer: no \n

In [22]:
# input tokenization
tokenizer.decode([1, 29871,    13, 21140,   340,   338,   385, 15278,   393, 16612,
           263,  3414, 29889, 14350,   263,  2933,   393,  7128,  2486,  1614,
          2167,   278,  2009, 29889,    13, 29871,    13,  2277, 29937,  2799,
          4080, 29901,    13,    13,    13,  1678,   596,  3414,   338,   304,
         22862,  5925,  5155,  1104,  1218,   304, 16083,  5354,   773,  4874,
         29914,  1217, 29914, 26026, 20890, 29889,   673,   408,  4874,   565,
           278,  3030, 11286,   278,  1139,  1919,  1234,   408,   694,   565,
           278,  3030,   947,   451,  2304,   278,  1139,  1919,  1234,   408,
          5505,   565,   278,  3030,   338,   451,  2821,  3307,   304,  1234,
           278,  1139, 29889,   539,    13,    13,  1678,   835,    13,   632,
            13,  1678, 15228, 29901, 10550, 21677, 26361,   756,  1063, 11395,
           284,   297,  3061, 19985,  1749,  8004,   310,  5164, 16021,  4235,
         29889,   360, 12099,   526, 10712,  7945,   519,   322,  5374,   519,
         15006, 29889,    13,    13,  1678,   894, 29901,  4683, 26361,  7945,
           519, 29973,    13,    13,  1678,   673, 29901,  4874,    13,    13,
          1678,   835,    13,    13,  1678, 15228, 29901, 10550, 21677, 26361,
           756,  1063, 11395,   284,   297,  3061, 19985,  1749,  8004,   310,
          5164, 16021,  4235, 29889,   360, 12099,   526, 10712,  7945,   519,
           322,  5374,   519, 15006, 29889,    13,    13,  1678,   894, 29901,
          1815, 26361,   367, 16370,   304,  6459,   322,  6655, 15724,   304,
           278, 10122,   310,  2702,   599, 15064,   575, 29973,    13,    13,
          1678,   673, 29901,  5505,    13,    13,  1678,   835,    13,    13,
          1678, 15228, 29901, 10550, 21677, 26361,   756,  1063, 11395,   284,
           297,  3061, 19985,  1749,  8004,   310,  5164, 16021,  4235, 29889,
           360, 12099,   526, 10712,  7945,   519,   322,  5374,   519, 15006,
         29889,    13,    13,  1678,   894, 29901,  4683,   360, 12099, 10712,
         22909,  1765,   287,   669, 21676,  1577,    13,    13,  1678,   673,
         29901,   694, 29871,    13,    13,  1678,   835,    13,    13,  1678,
         15228, 29901,    13,  1678,   319,  1206,   310,   805,   979,  1014,
         29881,  3631,   298,  4579,  4125,   313,  1799, 29928, 29950, 29897,
          1494,  1014,   279,   496,  1217,   333,  9736,   272, 19046,   482,
           313,  8132, 29950, 29897,  1363,   310,   263,  5796,   415,  2955,
          7463,  1559,   327,   333,   385,  5411,   952, 29885,   338,  5439,
         29889, 10506,   263,  1206,   756,  2360,  1063,  8967, 29889,   319,
         29871, 29945, 29906, 29899,  6360, 29899,  1025,  6114,  1090, 29893,
           296,   263,   274,   661, 24414, 16103,   363,   263,  5796,   415,
          2955,  7463,  1559,   327,   333,   385,  5411,   952, 29885, 29889,
           319, 15712,  6454,  5275, 12812, 10018,   393, 16698, 29950, 22856,
           758, 24130, 10835,   297,   278, 13446,   285, 20634,   322,  1014,
         29881,  3631,   298,  4579,  4125, 19540,   278,   274,   406, 12562,
           279, 12033,   272,  1974, 29889,   512, 26614,   354,  1052, 17517,
           310,   318,   307,  9089,   559, 29892,  6599, 17517,   310, 25995,
           566,   309, 17546,   305,  5095,   680, 29892,   322,  9126,   274,
           406,  6729,  1028,   979, 22576,   313,  9295, 29943, 29897,  3415,
         22061,   362,  3025,   274,   275,  1890,   270,  6038,   482,   892,
          8560,   408,   410, 11461,   433, 20009,  7539,  1860,   363, 19723,
          4705, 11625, 29889,  1551,   278, 25963,  1400,  3372,  1230,  2462,
         29892,   278, 16500, 15313,  1312,   310, 22261,  5224,  1250,   322,
           541,   517,   384,  6788, 29889,  3561,  1212,   293, 27396,   749,
          6382,   292, 10018,   263,  1014, 29881,  3631,   298,  4579,  4125,
           297,   278,   301,  3774,   359,   562,  1705,  5120, 29889,  8512,
           278,  4158,  2779,   471, 20607, 29892,   278, 16500, 10018,   694,
           452,  2192, 19227, 25828,  4835,   916,  1135,   278,  4560, 29288,
         29889,  2296,   471, 14914,  8976,  6703, 29889,   450,   298,  4579,
          4125, 23556,  1490, 22020,   322,   750, 22964,  3276,  6446, 29871,
         29896, 29945, 11405,  2678, 29889,  2439,  6788, 22020, 11684,  2618,
         29892,   322,  1183,   471,   766, 25389,   287, 29871, 29955, 11405,
          2678,  1728,   738,   452,  2192, 19227,   822,   293,   277, 29889,
            13,    13,  1678,   894, 29901,    13,  1678,  1706,   979,  1014,
         29881,  3631,   298,  4579,  4125, 29901,   263,  8617,  3100,   310,
           263,  5796,   415,  2955,   938,   945,   661,   616,   385,  5411,
           952, 29885, 29973,    13,    13,  1678,   673, 29901, 29871,    13,
           268,    13,   268,    13, 29871,    13,  2277, 29937, 13291, 29901,
            13])

'<s>\nBelow is an instruction that describes a task. Write a response that appropriately completes the request.\n \n### Instruction:\n\n\n    your task is to answering research questions relating to medical domain using yes/no/maybe responses. Answer as yes if the context supports the question , answer as no if the context does not support the question , answer as maybe if the context is not clear enough to answer the question.      \n\n    ###\n            \n    Context: Research involving dogs has been instrumental in advancing our understanding of various scientific fields. Dogs are highly trainable and sociable animals.\n\n    Question: Are dogs trainable?\n\n    Answer: yes\n\n    ###\n\n    Context: Research involving dogs has been instrumental in advancing our understanding of various scientific fields. Dogs are highly trainable and sociable animals.\n\n    Question: Can dogs be trained to detect and alert individuals to the presence of specific allergens?\n\n    Answer: maybe\n

In [59]:
tokenizer.decode([13])

'\n'

In [65]:
# long max_tokens =output
tokenizer.decode([    1, 29871,    13, 21140,   340,   338,   385, 15278,   393, 16612,
           263,  3414, 29889, 14350,   263,  2933,   393,  7128,  2486,  1614,
          2167,   278,  2009, 29889,    13, 29871,    13,  2277, 29937,  2799,
          4080, 29901,    13,    13,    13,  1678,   596,  3414,   338,   304,
         22862,  5925,  5155,  1104,  1218,   304, 16083,  5354,   773,  4874,
         29914,  1217, 29914, 26026, 20890, 29889,   673,   408,  4874,   565,
           278,  3030, 11286,   278,  1139,  1919,  1234,   408,   694,   565,
           278,  3030,   947,   451,  2304,   278,  1139,  1919,  1234,   408,
          5505,   565,   278,  3030,   338,   451,  2821,  3307,   304,  1234,
           278,  1139, 29889,   539,    13,    13,  1678,   835,    13,   632,
            13,  1678, 15228, 29901, 10550, 21677, 26361,   756,  1063, 11395,
           284,   297,  3061, 19985,  1749,  8004,   310,  5164, 16021,  4235,
         29889,   360, 12099,   526, 10712,  7945,   519,   322,  5374,   519,
         15006, 29889,    13,    13,  1678,   894, 29901,  4683, 26361,  7945,
           519, 29973,    13,    13,  1678,   673, 29901,  4874,    13,    13,
          1678,   835,    13,    13,  1678, 15228, 29901, 10550, 21677, 26361,
           756,  1063, 11395,   284,   297,  3061, 19985,  1749,  8004,   310,
          5164, 16021,  4235, 29889,   360, 12099,   526, 10712,  7945,   519,
           322,  5374,   519, 15006, 29889,    13,    13,  1678,   894, 29901,
          1815, 26361,   367, 16370,   304,  6459,   322,  6655, 15724,   304,
           278, 10122,   310,  2702,   599, 15064,   575, 29973,    13,    13,
          1678,   673, 29901,  5505,    13,    13,  1678,   835,    13,    13,
          1678, 15228, 29901, 10550, 21677, 26361,   756,  1063, 11395,   284,
           297,  3061, 19985,  1749,  8004,   310,  5164, 16021,  4235, 29889,
           360, 12099,   526, 10712,  7945,   519,   322,  5374,   519, 15006,
         29889,    13,    13,  1678,   894, 29901,  4683,   360, 12099, 10712,
         22909,  1765,   287,   669, 21676,  1577,    13,    13,  1678,   673,
         29901,   694, 29871,    13,    13,  1678,   835,    13,    13,  1678,
         15228, 29901,    13,  1678,   319,  1206,   310,   805,   979,  1014,
         29881,  3631,   298,  4579,  4125,   313,  1799, 29928, 29950, 29897,
          1494,  1014,   279,   496,  1217,   333,  9736,   272, 19046,   482,
           313,  8132, 29950, 29897,  1363,   310,   263,  5796,   415,  2955,
          7463,  1559,   327,   333,   385,  5411,   952, 29885,   338,  5439,
         29889, 10506,   263,  1206,   756,  2360,  1063,  8967, 29889,   319,
         29871, 29945, 29906, 29899,  6360, 29899,  1025,  6114,  1090, 29893,
           296,   263,   274,   661, 24414, 16103,   363,   263,  5796,   415,
          2955,  7463,  1559,   327,   333,   385,  5411,   952, 29885, 29889,
           319, 15712,  6454,  5275, 12812, 10018,   393, 16698, 29950, 22856,
           758, 24130, 10835,   297,   278, 13446,   285, 20634,   322,  1014,
         29881,  3631,   298,  4579,  4125, 19540,   278,   274,   406, 12562,
           279, 12033,   272,  1974, 29889,   512, 26614,   354,  1052, 17517,
           310,   318,   307,  9089,   559, 29892,  6599, 17517,   310, 25995,
           566,   309, 17546,   305,  5095,   680, 29892,   322,  9126,   274,
           406,  6729,  1028,   979, 22576,   313,  9295, 29943, 29897,  3415,
         22061,   362,  3025,   274,   275,  1890,   270,  6038,   482,   892,
          8560,   408,   410, 11461,   433, 20009,  7539,  1860,   363, 19723,
          4705, 11625, 29889,  1551,   278, 25963,  1400,  3372,  1230,  2462,
         29892,   278, 16500, 15313,  1312,   310, 22261,  5224,  1250,   322,
           541,   517,   384,  6788, 29889,  3561,  1212,   293, 27396,   749,
          6382,   292, 10018,   263,  1014, 29881,  3631,   298,  4579,  4125,
           297,   278,   301,  3774,   359,   562,  1705,  5120, 29889,  8512,
           278,  4158,  2779,   471, 20607, 29892,   278, 16500, 10018,   694,
           452,  2192, 19227, 25828,  4835,   916,  1135,   278,  4560, 29288,
         29889,  2296,   471, 14914,  8976,  6703, 29889,   450,   298,  4579,
          4125, 23556,  1490, 22020,   322,   750, 22964,  3276,  6446, 29871,
         29896, 29945, 11405,  2678, 29889,  2439,  6788, 22020, 11684,  2618,
         29892,   322,  1183,   471,   766, 25389,   287, 29871, 29955, 11405,
          2678,  1728,   738,   452,  2192, 19227,   822,   293,   277, 29889,
            13,    13,  1678,   894, 29901,    13,  1678,  1706,   979,  1014,
         29881,  3631,   298,  4579,  4125, 29901,   263,  8617,  3100,   310,
           263,  5796,   415,  2955,   938,   945,   661,   616,   385,  5411,
           952, 29885, 29973,    13,    13,  1678,   673, 29901, 29871,    13,
           268,    13,   268,    13, 29871,    13,  2277, 29937, 13291, 29901,
            13,    13,    13,    13,    13,    13,    13,    13,    13,    13,
            13])

'<s>\nBelow is an instruction that describes a task. Write a response that appropriately completes the request.\n \n### Instruction:\n\n\n    your task is to answering research questions relating to medical domain using yes/no/maybe responses. Answer as yes if the context supports the question , answer as no if the context does not support the question , answer as maybe if the context is not clear enough to answer the question.      \n\n    ###\n            \n    Context: Research involving dogs has been instrumental in advancing our understanding of various scientific fields. Dogs are highly trainable and sociable animals.\n\n    Question: Are dogs trainable?\n\n    Answer: yes\n\n    ###\n\n    Context: Research involving dogs has been instrumental in advancing our understanding of various scientific fields. Dogs are highly trainable and sociable animals.\n\n    Question: Can dogs be trained to detect and alert individuals to the presence of specific allergens?\n\n    Answer: maybe\n

In [72]:
tokenizer.decode([268])

'   '